### Table of Contents

# 1. Import Data

## 1.1 Import the needed libraries

In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from datetime import date
from scipy.stats import zscore

%matplotlib inline
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

## 1.2 Import a integrate data

In [129]:
df_crm = pd.read_csv('crm.csv')
df_mkt = pd.read_csv('mkt.csv')
df_sales = pd.read_excel('sales.xlsx')

In [130]:
df = pd.merge(pd.merge(df_crm,df_sales,on='CustomerID',how="inner"),df_mkt,on="CustomerID",how="inner")

## 1.3 Set Index


In [131]:
df.set_index('CustomerID',inplace = True)

## 1.4 Check and removing duplicates

In [132]:
df[df.duplicated()] # checking duplicates

,Name,Birthyear,Education,Marital_Status,Income,Kid_Younger6,Children_6to18,Date_Adherence,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Response_Cmp1,Response_Cmp2,Response_Cmp3,Response_Cmp4,Response_Cmp5
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,
32924,Mrs. Wanda Terry,1986,Graduation,Divorced,41264.0,1,1,2020-10-03 00:00:00,61.0,70.0,20,165.0,40.0,0.0,7.0,2,3,1,3,7,0,0,0,0,0,0
8276,Mrs. Mary Jackson,1968,HighSchool,Married,118238.0,0,0,2020-10-13 00:00:00,42.0,5940.0,1002,9465.0,1100.0,1100.0,32.0,1,6,7,8,2,0,0,0,0,0,0
32966,Mr. Steven Baker,1961,HighSchool,Married,84974.0,0,1,2022-02-17 00:00:00,56.0,2610.0,840,2160.0,840.0,960.0,133.0,3,6,5,9,3,0,0,0,0,0,0
5627,Mr. Jonathan Clark,1988,Graduation,Married,40975.0,1,0,2022-04-28 00:00:00,31.0,130.0,0,120.0,20.0,20.0,5.0,1,3,1,4,3,0,0,0,0,0,0
29357,Mr. Eric Springer,1972,PhD,Married,77778.0,0,1,2022-04-19 00:00:00,92.0,1240.0,60,315.0,20.0,20.0,33.0,1,5,2,4,6,0,0,1,0,0,0
26891,Mr. Sam Peake,1992,Graduation,Married,74147.0,1,0,2022-03-21 00:00:00,13.0,610.0,320,915.0,20.0,240.0,78.0,2,6,3,2,6,0,1,0,0,0,0
21269,Mr. Nathan Glover,1971,Graduation,Married,91241.0,0,1,2022-03-28 00:00:00,18.0,1780.0,220,2370.0,540.0,380.0,59.0,1,8,3,6,5,0,0,0,0,0,0
32777,Mr. Joe James,1967,Master,Married,101251.0,0,1,2021-12-30 00:00:00,26.0,7350.0,2780,4170.0,500.0,1760.0,66.0,2,8,8,6,3,0,1,0,0,0,0
26285,Mr. Oliver Sanderson,1986,Basic,Together,22345.0,1,0,2021-03-21 00:00:00,54.0,50.0,180,285.0,240.0,280.0,39.0,3,5,2,2,9,0,1,0,0,0,0


In [133]:
df = df[~df.duplicated()] # drop duplicates rows

# 2. Explore Data

## 2.0 Data profiling

Se não quiserem instalar a biblioteca não corram esta secção. Caso contrário o comando para instalar é pip install ydata-profiling. No final **apagar esta secção**

In [134]:
#from ydata_profiling import ProfileReport
#profile= ProfileReport (df, title= "DSML_Project")

In [135]:
#profile.to_file('DSML_profile.html')

## 2.1 Basic Exploration

Q: _To check the number of columns and rows_ we used `shape` _attribute_

In [136]:
df.shape

(7000, 26)

> A: _The dataset has **7000 rows** and **26 columns**_

__*Q*__: Check the name of the features of the dataset we used `columns` _attribute_

In [137]:
df.columns

Index(['Name', 'Birthyear', 'Education', 'Marital_Status', 'Income',
       'Kid_Younger6', 'Children_6to18', 'Date_Adherence', 'Recency',
       'MntMeat&Fish', 'MntEntries', 'MntVegan&Vegetarian', 'MntDrinks',
       'MntDesserts', 'MntAdditionalRequests', 'NumOfferPurchases',
       'NumAppPurchases', 'NumTakeAwayPurchases', 'NumStorePurchases',
       'NumAppVisitsMonth', 'Complain', 'Response_Cmp1', 'Response_Cmp2',
       'Response_Cmp3', 'Response_Cmp4', 'Response_Cmp5'],
      dtype='object')

> A: The dataset has the following columns/features names: <br>
        >Index. CustomerID
        >1. 'Name' <br>
        >2. 'Birthyear'<br>
        >3. 'Education'<br>
        >4. 'Marital_Status'<br>
        >5. 'Income'<br>
        >6. 'Kid_Younger6'<br>
        >7. 'Children_6to18'<br>
        >8. 'Date_Adherence'<br>
        >9. 'Recency'<br>
        >10. 'MntMeat&Fish'<br>
        >11. 'MntEntries'<br>
        >12. 'MntVegan&Vegetarian'<br>
        >13. 'MntDrinks'<br>
        >14. 'MntDesserts'<br>
        >15. 'MntAdditionalRequests'<br>
        >16. 'NumOfferPurchases'<br>
        >17. 'NumAppPurchases'<br>
        >18. 'NumTakeAwayPurchases'<br>
        >19. 'NumStorePurchases'<br>
        >20. 'NumAppVisitsMonth'<br>
        >21. 'Complain'<br>
        >22. 'Response_Cmp1'<br>
        >23. 'Response_Cmp2'<br>
        >24. 'Response_Cmp3'<br>
        >25. 'Response_Cmp4'<br>
        >26. 'Response_Cmp5'<br>

Q: First glance of the dataset using `head` and `tail` methods to check the first and last 5 rows.

In [138]:
df.head(3)

,Name,Birthyear,Education,Marital_Status,Income,Kid_Younger6,Children_6to18,Date_Adherence,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Response_Cmp1,Response_Cmp2,Response_Cmp3,Response_Cmp4,Response_Cmp5
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,
14594,Miss Leah Lewis,1981,Graduation,Single,113947.0,0,0,2021-01-09 00:00:00,43.0,4640.0,3380,5700.0,2240.0,562.0,154.0,2,10,7,7,4,0,0,1,0,0,0
19070,Miss Grace Black,1974,Graduation,Together,114297.0,0,0,2021-04-10 00:00:00,74.0,7080.0,3680,7620.0,1840.0,1220.0,46.0,1,5,9,12,2,0,0,0,0,0,0
29888,Mr. Gavin Morrison,1960,PhD,Together,115476.0,0,0,2022-01-12 00:00:00,85.0,4130.0,380,4890.0,3640.0,380.0,28.0,1,5,5,7,1,0,0,0,0,0,0


In [139]:
df.tail(3)

,Name,Birthyear,Education,Marital_Status,Income,Kid_Younger6,Children_6to18,Date_Adherence,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Response_Cmp1,Response_Cmp2,Response_Cmp3,Response_Cmp4,Response_Cmp5
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,
29186,Mr. Thomas Churchill,1967,PhD,Together,56039.0,0,1,2021-12-15 00:00:00,30.0,540.0,0,135.0,0.0,0.0,5.0,2,3,1,4,6,0,0,0,0,0,0
23312,Miss Angela Murray,1989,Graduation,Single,44867.0,1,0,2020-11-12 00:00:00,8.0,80.0,120,210.0,120.0,100.0,17.0,1,3,1,3,8,0,0,0,0,0,0
15581,Miss Diana Taylor,1960,Graduation,Single,124379.0,0,0,2021-02-04 00:00:00,80.0,7280.0,422,1692.0,3760.0,1880.0,235.0,1,6,12,11,2,0,0,0,1,1,0


Q: To check the basic information of the dataset we've used the `info` method

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 14594 to 15581
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   7000 non-null   object 
 1   Birthyear              7000 non-null   int64  
 2   Education              6986 non-null   object 
 3   Marital_Status         7000 non-null   object 
 4   Income                 7000 non-null   float64
 5   Kid_Younger6           7000 non-null   int64  
 6   Children_6to18         7000 non-null   int64  
 7   Date_Adherence         7000 non-null   object 
 8   Recency                6977 non-null   float64
 9   MntMeat&Fish           7000 non-null   float64
 10  MntEntries             7000 non-null   int64  
 11  MntVegan&Vegetarian    7000 non-null   float64
 12  MntDrinks              6972 non-null   float64
 13  MntDesserts            7000 non-null   float64
 14  MntAdditionalRequests  7000 non-null   float64
 15 

>A: We can observe the data type of the dataset and the how many of features per data type  `dtypes: float64 - (7), int64 - (15), object - (4)`, the memory usage of `1.4+MB`, and the non-null values present per columns. <br>
> Using only `info` method we understand that `'Education', 'Recency', 'MntDrinks'` have __14, 23, 28 null values__ that require some action.

# 2.2 Statistical Exploration

## 2.2.1 Numerical Variables

In [141]:
df.describe()

,Birthyear,Income,Kid_Younger6,Children_6to18,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Response_Cmp1,Response_Cmp2,Response_Cmp3,Response_Cmp4,Response_Cmp5
count,7000.000000,7000.000000,7000.000000,7000.000000,6977.000000,7000.000000,7000.000000,7000.000000,6972.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,1976.451429,77988.962407,0.440571,0.490571,49.235058,3079.523800,534.749429,2785.050786,545.657544,540.656029,42.556186,2.448429,6.015714,3.834571,5.790571,5.278286,0.010286,0.079143,0.073286,0.083000,0.065857,0.014286
std,11.996270,35409.810253,0.543477,0.542174,28.922688,3370.377166,787.846684,3908.718244,805.149088,802.221866,49.650747,2.306968,2.745537,3.331142,3.295708,2.748596,0.100903,0.269981,0.260624,0.275902,0.248050,0.118675
min,1948.000000,2493.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1967.000000,51586.250000,0.000000,0.000000,24.000000,250.000000,40.000000,240.000000,40.000000,40.000000,9.000000,1.000000,4.000000,1.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1977.000000,77190.000000,0.000000,0.000000,49.000000,1820.000000,180.000000,1110.000000,180.000000,180.000000,24.000000,2.000000,6.000000,3.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1985.000000,102016.250000,1.000000,1.000000,74.000000,5070.000000,680.000000,3795.000000,700.000000,680.000000,57.000000,3.000000,8.000000,5.000000,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2005.000000,237639.725000,2.000000,2.000000,99.000000,14980.000000,3980.000000,25974.000000,3980.000000,3980.000000,249.000000,16.000000,13.000000,24.000000,13.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


> _The describe return we can get a first glance and make some conclusion:_

>__Birthyear__ - could originate an Age column for readability purposes<br>
__Income__ - Min and Max are very far from each other and far from the mean value which could indicate outliers<br>
__Recency__ - 6977 valid values, hence we should look in deep and decide on how to minimize that effect of missing values<br>
__MntMeat&Fish__ - Min and Max are distant from each other and have high standard deviation which could effect some future conclusion<br>
__MntEntries__ - Again has high standard deviation that we should analyze, Min and Max far apart, similar to MntMeat&Fish<br>
__MntVegan&Vegetarian__ - Similar to the previous two Mnt columns<br>
__MntDrinks, MntDesserts__ - Seems to be very similar between them<br>
__MntAdditionalRequests__ - The max value standard deviation seems high and also the max value very far apart from the mean<br>
__NumOfferPurchases, NumTakeAwayPurchases, NumAppVisitsMonth__  - Have a max value to distante from the mean that could be true but we need to take into account<br>
__NumAppPurchases, SumStorePurchases__ - Seems does not have strange summary statistcs<br>
__Kid_Younger6, Children_6to18__ - 75% of clients have at least one child

**Q**: Skewness of each variable 

In [142]:
df.skew()

/var/folders/gt/lg95g0n95mqdr3vk5d1k062h0000gn/T/ipykernel_37366/547062910.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.skew()


Birthyear               -0.092478
Income                   0.844721
Kid_Younger6             0.693587
Children_6to18           0.451693
Recency                  0.017676
MntMeat&Fish             1.148862
MntEntries               2.087220
MntVegan&Vegetarian      2.486897
MntDrinks                2.046308
MntDesserts              2.057811
MntAdditionalRequests    1.826522
NumOfferPurchases        2.860514
NumAppPurchases          0.525873
NumTakeAwayPurchases     2.254158
NumStorePurchases        0.623254
NumAppVisitsMonth        1.005192
Complain                 9.709429
Response_Cmp1            3.118572
Response_Cmp2            3.275500
Response_Cmp3            3.023677
Response_Cmp4            3.501448
Response_Cmp5            8.187993
dtype: float64

Concerning the variables' skewness, we can conclude the following:
- `Moderate skewness (between |0.5| and |1.0|)`: Birthyear, Income, Kid_Younger6, Children_6to18, Recency, NumAppPurchases, NumStorePurchases, NumAppVisitsMonth
- `High skewness (higher than |1.0|)`: MntMeat&Fish, MntEntries, MntVegan&Vegetarian, MntDrinks, MntDesserts, MntAdditionalRequests, NumOfferPurchases, NumTakeAwayPurchases, Complain, Response_Cmp1, Response_Cmp2, Response_Cmp3, Response_Cmp4, Response_Cmp5

In [143]:
df.kurt()

/var/folders/gt/lg95g0n95mqdr3vk5d1k062h0000gn/T/ipykernel_37366/3952544226.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.kurt()


Birthyear                -0.857341
Income                    2.649035
Kid_Younger6             -0.651127
Children_6to18           -0.973333
Recency                  -1.197978
MntMeat&Fish              0.532098
MntEntries                4.096167
MntVegan&Vegetarian       8.431538
MntDrinks                 3.839256
MntDesserts               3.813464
MntAdditionalRequests     3.084190
NumOfferPurchases        10.997946
NumAppPurchases          -0.222899
NumTakeAwayPurchases      8.582138
NumStorePurchases        -0.694327
NumAppVisitsMonth         4.992870
Complain                 92.299387
Response_Cmp1             7.727700
Response_Cmp2             8.731398
Response_Cmp3             7.144665
Response_Cmp4            10.263068
Response_Cmp5            65.061813
dtype: float64

Features with kurtosis higher than 3 could indicate presence of outliers, hence we should have special considerantion with the following features:
>MntEntries, MntVegan&Vegetarian, MntDrinks, MntDesserts, NumOfferPurchases, NumAppVisitsMonth

Note: Binomial Variables Complain, and Response_Cmp1 the kurtosis we will not consider as outliers

## 2.2.2 Categorical Values

In [144]:
df.describe(include = object)

,Name,Education,Marital_Status,Date_Adherence
count,7000,6986,7000,7000
unique,6241,9,10,701
top,Mr. Stewart Grant,Graduation,Married,2020-09-19 00:00:00
freq,3,3497,2830,23


> We can conclude that the education as **14 missing** values

#### Level/Possible values of Categorical Features

### `Name` prefix unique values and count

In [145]:
df['Name'].str.partition(" ")[0].value_counts()

Mr.     5133
Miss    1469
Mrs.     398
Name: 0, dtype: int64

With the prefix we can generate a `gender` feature to further explore the dataset. We will deal with that in the data transformation capther

#### **`Gender`** feature creation

In [146]:
df["Gender"] = df['Name'].str.partition(" ")[0]
df = df.replace({"Gender":{"Mr.": 1,"Miss": 0,"Mrs.": 0}})

### `Education` unique values and count

In [147]:
df["Education"].value_counts()

Graduation    3497
PhD           1494
Master        1135
HighSchool     663
Basic          179
master           7
graduation       7
phd              2
highschool       2
Name: Education, dtype: int64

We have some issues that will need trasformatioin:<br>
- Graduation, Master, HighSchool are written in different ways<br>
- `Basic` and `HighSchool` need different levels?

#### Education standardization

In [148]:
df = df.replace({"Education":{"master":"Master", "graduation":"Graduation", "phd":"PhD","highschool":"HighSchool"}})

### `Marital_Status` unique values and count

In [149]:
df["Marital_Status"].value_counts()

Married     2830
Together    1683
Single      1525
Divorced     637
Widow        243
married       36
together      23
single        13
divorced       8
widow          2
Name: Marital_Status, dtype: int64

Similarly to previous feature we also have some issues that need transformation:<br>
- Married, Together, Single, Divorced and Widow are written with lower and capital letters
- We could also consider that Married and Together are similar and joined them in the same level<br>

#### Marital_Status standardization

In [150]:
df = df.replace({"Marital_Status":{"married":"Married", "together":"Married", "single":"Single","widow":"Widow","divorced":"Divorced","Together":"Married"}})
df["Marital_Status"].value_counts()

Married     4572
Single      1538
Divorced     645
Widow        245
Name: Marital_Status, dtype: int64

`Date_Adherence` unqiue values and count

In [151]:
df["Date_Adherence"].value_counts()

2020-09-19    23
2021-08-22    20
2021-07-30    20
2022-05-23    20
2021-08-21    19
              ..
2021-03-09     3
2020-09-18     2
2020-07-30     2
2020-08-29     2
2021-03-22     2
Name: Date_Adherence, Length: 701, dtype: int64

`Date_Adherence` is a date and will need transformation to a date format for further exploration

## 2.3 Visual Exploration

### 2.3.1 Numerical Variables

## 2.4. In-Depth Exploration

# 3. Preprocess Data

## 3.1. Data Cleaning

### 3.1.1. Outliers

In [152]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
#ax1.boxplot(df['MntVegan&Vegetarian'])
#ax2.boxplot(df['Income'])

In [153]:
df.drop(df[abs(zscore(df['MntVegan&Vegetarian'])) > 3].index, inplace=True)
df.drop(df[abs(zscore(df['Income'])) > 3].index, inplace=True)

In [154]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
#ax1.boxplot(df['MntVegan&Vegetarian'])
#ax2.boxplot(df['Income'])

## 3.1.2 Skewness Correction

In [155]:
df['MntMeat&Fish'] = df['MntMeat&Fish'].apply(lambda x: np.log10(x + 1))
df['MntVegan&Vegetarian'] = df['MntVegan&Vegetarian'].apply(lambda x: np.log10(x + 1))
df['MntEntries'] = df['MntEntries'].apply(lambda x: np.log10(x + 1))
df['MntDrinks'] = df['MntDrinks'].apply(lambda x: np.log10(x + 1))
df['MntDesserts'] = df['MntDesserts'].apply(lambda x: np.log10(x + 1))
df['MntAdditionalRequests'] = df['MntAdditionalRequests'].apply(lambda x: np.log10(x + 1))
df['NumOfferPurchases'] = df['NumOfferPurchases'].apply(lambda x: np.log10(x + 1))
df['NumTakeAwayPurchases'] = df['NumTakeAwayPurchases'].apply(lambda x: np.log10(x + 1))
df['Complain'] = df['Complain'].apply(lambda x: np.log10(x + 1))
df['Response_Cmp1'] = df['Response_Cmp1'].apply(lambda x: np.log10(x + 1))
df['Response_Cmp2'] = df['Response_Cmp2'].apply(lambda x: np.log10(x + 1))
df['Response_Cmp3'] = df['Response_Cmp3'].apply(lambda x: np.log10(x + 1))
df['Response_Cmp4'] = df['Response_Cmp4'].apply(lambda x: np.log10(x + 1))
df['Response_Cmp5'] = df['Response_Cmp5'].apply(lambda x: np.log10(x + 1))

### 3.1.2. Missing Values

In [156]:
Response_is_null = df["Response_Cmp1"].isna().sum() + df["Response_Cmp2"].isna().sum() + df["Response_Cmp3"].isna().sum() + df["Response_Cmp4"].isna().sum()
Response_is_null == 0

True

In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6884 entries, 14594 to 15581
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   6884 non-null   object 
 1   Birthyear              6884 non-null   int64  
 2   Education              6870 non-null   object 
 3   Marital_Status         6884 non-null   object 
 4   Income                 6884 non-null   float64
 5   Kid_Younger6           6884 non-null   int64  
 6   Children_6to18         6884 non-null   int64  
 7   Date_Adherence         6884 non-null   object 
 8   Recency                6861 non-null   float64
 9   MntMeat&Fish           6884 non-null   float64
 10  MntEntries             6884 non-null   float64
 11  MntVegan&Vegetarian    6884 non-null   float64
 12  MntDrinks              6856 non-null   float64
 13  MntDesserts            6884 non-null   float64
 14  MntAdditionalRequests  6884 non-null   float64
 15 

In [158]:
df.isna().sum()

Name                      0
Birthyear                 0
Education                14
Marital_Status            0
Income                    0
Kid_Younger6              0
Children_6to18            0
Date_Adherence            0
Recency                  23
MntMeat&Fish              0
MntEntries                0
MntVegan&Vegetarian       0
MntDrinks                28
MntDesserts               0
MntAdditionalRequests     0
NumOfferPurchases         0
NumAppPurchases           0
NumTakeAwayPurchases      0
NumStorePurchases         0
NumAppVisitsMonth         0
Complain                  0
Response_Cmp1             0
Response_Cmp2             0
Response_Cmp3             0
Response_Cmp4             0
Response_Cmp5             0
Gender                    0
dtype: int64

- **`Education`**, **`Recency`**, **`MntDrinks`** and **`MntTotal`** (due to dependancy of `MntDrinks`) have missing values

#### Filling the missing values

Fill `Education` with the mode

In [159]:
df["Education"].fillna(df["Education"].mode()[0], inplace = True)

Fill `Recency` with the median value

In [160]:
df["Recency"].fillna(df["Recency"].mean(), inplace = True)

In [161]:
df_mnt = df[[ 'MntMeat&Fish', 'MntEntries', 'MntVegan&Vegetarian', 'MntDrinks',
       'MntDesserts', 'MntAdditionalRequests']]

imputer = KNNImputer(n_neighbors=3)
array_impute = imputer.fit_transform(df_mnt)
df_mnt = pd.DataFrame(array_impute, columns = df_mnt.columns)

In [162]:
df["MntDrinks"] = df_mnt["MntDrinks"].values

In [163]:
df["MntTotal"] = df['MntMeat&Fish'] + df['MntEntries'] + df['MntVegan&Vegetarian'] + df['MntDrinks'] + df['MntDesserts']

In [164]:
df.isna().sum()

Name                     0
Birthyear                0
Education                0
Marital_Status           0
Income                   0
Kid_Younger6             0
Children_6to18           0
Date_Adherence           0
Recency                  0
MntMeat&Fish             0
MntEntries               0
MntVegan&Vegetarian      0
MntDrinks                0
MntDesserts              0
MntAdditionalRequests    0
NumOfferPurchases        0
NumAppPurchases          0
NumTakeAwayPurchases     0
NumStorePurchases        0
NumAppVisitsMonth        0
Complain                 0
Response_Cmp1            0
Response_Cmp2            0
Response_Cmp3            0
Response_Cmp4            0
Response_Cmp5            0
Gender                   0
MntTotal                 0
dtype: int64

## 3.2. Data Transformation

### 3.2.1. Create new Variables

### Utils

#### Creating Age variable from the Birthyear

In [165]:
df['Age'] = df.Birthyear.apply(lambda x: date.today().year-x)

In [166]:
df.drop('Birthyear', axis= 1, inplace = True)

In [167]:
#### Creating card adherence age variable from the Date adherence

In [168]:
from datetime import datetime
df = df.replace({"Date_Adherence":{"2/29/2022": datetime.strptime("2022-03-01", '%Y-%m-%d')}}) #2022 is not a leap year, therefore 29/02/2022 is not a possible day

In [169]:
df['daysAsCardClient'] = df['Date_Adherence'].apply(lambda x: (date.today() - x.date()).days)

In [170]:
df.drop('Date_Adherence', axis= 1, inplace = True)

#### Fill Education

In [171]:
edu_encode = pd.get_dummies(df.Education, drop_first= True)
df = pd.concat([df, edu_encode], axis = 1)

In [172]:
df.drop('Education', axis = 1, inplace = True)

#### Fill Maritial Status

In [173]:
marital_encode = pd.get_dummies(df.Marital_Status, drop_first= True)
df = pd.concat([df, marital_encode], axis = 1)

In [174]:
df.drop('Marital_Status', axis= 1, inplace = True)

#### Create MntTotal

In [175]:
df["MntTotal"] = df['MntMeat&Fish'] + df['MntEntries'] + df['MntVegan&Vegetarian'] + df['MntDrinks'] + df['MntDesserts'] + df['MntAdditionalRequests']
df["MntTotal"]
# em falta Mnt Add Requests

CustomerID
14594    19.242890
19070    19.321940
29888    17.490921
12221    12.145689
21107    18.204269
           ...    
30686    14.183984
17210    16.426555
29186     5.644887
23312    11.657932
15581    18.939792
Name: MntTotal, Length: 6884, dtype: float64

#### Create Mnt Pday Card

In [176]:
df['Mnt_pday_card']= df.MntTotal/df.daysAsCardClient

#### Create Response Campaigns

In [177]:
df['Response_Campaigns'] = df['Response_Cmp1'] + df['Response_Cmp2'] + df['Response_Cmp3'] + df['Response_Cmp4'] + df[
       'Response_Cmp5']

In [178]:
df.drop(['Response_Cmp1', 'Response_Cmp2', 'Response_Cmp3', 'Response_Cmp4', 'Response_Cmp5'], axis=1, inplace=True)

#### Create Total Kids

In [179]:
df["Total_Kids"] = df["Kid_Younger6"] + df["Children_6to18"]

#### Create Has Kids

In [180]:
df["has_Kids"] = df["Total_Kids"].apply(lambda x: 0 if x == 0 else 1)
df["has_Kids"]

CustomerID
14594    0
19070    0
29888    0
12221    1
21107    0
        ..
30686    0
17210    0
29186    1
23312    1
15581    0
Name: has_Kids, Length: 6884, dtype: int64

In [181]:
df.drop(['Kid_Younger6', 'Children_6to18'], axis = 1, inplace = True)

In [182]:
df["age_bins"] = pd.cut(df["Age"], bins = 5)

In [183]:
df["age_bins"] # corrigir para permitir pca

CustomerID
14594      (40.8, 52.2]
19070      (40.8, 52.2]
29888      (52.2, 63.6]
12221      (40.8, 52.2]
21107      (40.8, 52.2]
              ...      
30686    (17.943, 29.4]
17210      (29.4, 40.8]
29186      (52.2, 63.6]
23312      (29.4, 40.8]
15581      (52.2, 63.6]
Name: age_bins, Length: 6884, dtype: category
Categories (5, interval[float64, right]): [(17.943, 29.4] < (29.4, 40.8] < (40.8, 52.2] < (52.2, 63.6] < (63.6, 75.0]]

## Incoherencies


In [184]:
df[(df[['MntMeat&Fish', 'MntEntries',
        'MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts',
        'MntAdditionalRequests']].sum(axis = 1) > 0) & (df[['NumAppPurchases', 'NumTakeAwayPurchases', 'NumStorePurchases']].sum(axis = 1) <= 0)]

,Name,Income,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Gender,MntTotal,Age,daysAsCardClient,Graduation,HighSchool,Master,PhD,Married,Single,Widow,Mnt_pday_card,Response_Campaigns,Total_Kids,has_Kids,age_bins
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6185,Mr. Matt Clarkson,4609.6,32.0,1.301030,1.041393,1.872156,1.041393,1.462398,0.623249,1.20412,0,0.0,0,20,0.00000,1,7.341619,32,823,0,0,1,0,1,0,0,0.008921,0.00000,2,1,"(29.4, 40.8]"
5318,Mr. Liam MacDonald,3005.4,16.0,0.778151,0.954243,0.929419,0.845098,0.698970,0.301030,0.00000,0,0.0,0,20,0.00000,1,4.506911,55,805,0,1,0,0,1,0,0,0.005599,0.00000,0,0,"(52.2, 63.6]"
25079,Mr. Harry Dickens,5048.0,64.0,1.146128,0.698970,0.740363,0.000000,0.477121,0.000000,1.20412,0,0.0,0,20,0.00000,1,3.062582,64,942,0,0,0,1,1,0,0,0.003251,0.00000,1,1,"(63.6, 75.0]"
23297,Mr. Harry Newman,3909.6,29.0,1.113943,0.000000,0.740363,0.000000,0.000000,0.113943,1.20412,0,0.0,0,20,0.00000,1,1.968249,43,844,1,0,0,0,1,0,0,0.002332,0.00000,2,1,"(40.8, 52.2]"
34565,Miss Jasmine Russell,7644.8,15.0,1.568202,0.477121,1.966142,1.799341,1.591065,1.152288,1.20412,0,0.0,0,19,0.00000,0,8.554158,61,652,0,1,0,0,0,1,0,0.013120,0.30103,2,1,"(52.2, 63.6]"
5108,Mr. Sebastian Abraham,10724.2,93.0,2.457882,2.392697,2.583765,2.143015,1.886491,0.518514,0.00000,0,0.0,0,19,0.00000,1,11.982364,33,400,0,0,0,1,1,0,0,0.029956,0.00000,0,0,"(29.4, 40.8]"
18872,Miss Rachel Young,3550.0,4.0,1.544068,0.000000,1.342423,0.698970,0.477121,0.079181,0.00000,0,0.0,0,20,0.00000,0,4.141763,40,648,0,0,0,1,0,1,0,0.006392,0.00000,1,1,"(29.4, 40.8]"
13388,Mr. Eric Berry,4554.8,99.0,0.778151,1.322219,1.311754,0.698970,1.230449,0.414973,1.20412,0,0.0,0,20,0.00000,1,5.756517,39,808,0,1,0,0,1,0,0,0.007124,0.00000,1,1,"(29.4, 40.8]"
14084,Mr. Christopher MacLeod,5281.3,17.0,1.079181,1.041393,1.568202,0.845098,1.041393,0.146128,0.00000,0,0.0,0,19,0.00000,1,5.721394,29,354,1,0,0,0,0,1,0,0.016162,0.00000,1,1,"(17.943, 29.4]"


In [185]:
# como justificar a atribuição de [0,0,0,0,0,0] às variaveis de monetary?

df.loc[(df[['MntMeat&Fish', 'MntEntries',
            'MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts',
            'MntAdditionalRequests']].sum(axis = 1) > 0) & (df[['NumAppPurchases', 'NumTakeAwayPurchases', 'NumStorePurchases']].sum(axis = 1) <= 0),['MntMeat&Fish','MntEntries','MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts','MntAdditionalRequests']] = [0,0,0,0,0,0]

In [186]:
df[(df[['MntMeat&Fish', 'MntEntries',
        'MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts',
        'MntAdditionalRequests']].sum(axis = 1) > 0) & (df[['NumAppPurchases', 'NumTakeAwayPurchases', 'NumStorePurchases']].sum(axis = 1) <= 0)] # confirmação do ajuste

,Name,Income,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Gender,MntTotal,Age,daysAsCardClient,Graduation,HighSchool,Master,PhD,Married,Single,Widow,Mnt_pday_card,Response_Campaigns,Total_Kids,has_Kids,age_bins
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [187]:
df[(df['NumOfferPurchases'] > df[['NumAppPurchases','NumTakeAwayPurchases','NumStorePurchases']].sum(axis=1))]

,Name,Income,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Gender,MntTotal,Age,daysAsCardClient,Graduation,HighSchool,Master,PhD,Married,Single,Widow,Mnt_pday_card,Response_Campaigns,Total_Kids,has_Kids,age_bins
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6185,Mr. Matt Clarkson,4609.6,32.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.20412,0,0.0,0,20,0.0,1,7.341619,32,823,0,0,1,0,1,0,0,0.008921,0.00000,2,1,"(29.4, 40.8]"
12011,Mr. Keith Mills,8503.6,70.0,3.019947,0.000000,1.511883,0.000000,0.000000,1.143015,1.20412,0,0.0,1,20,0.0,1,5.674845,56,403,1,0,0,0,1,0,0,0.014082,0.90309,0,0,"(52.2, 63.6]"
31391,Mr. Justin Forsyth,9709.1,26.0,2.484300,2.267172,2.425697,2.232996,1.995635,0.770852,1.20412,0,0.0,1,19,0.0,1,12.176652,65,432,0,0,0,1,0,0,1,0.028187,0.00000,0,0,"(63.6, 75.0]"
25079,Mr. Harry Dickens,5048.0,64.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.20412,0,0.0,0,20,0.0,1,3.062582,64,942,0,0,0,1,1,0,0,0.003251,0.00000,1,1,"(63.6, 75.0]"
23297,Mr. Harry Newman,3909.6,29.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.20412,0,0.0,0,20,0.0,1,1.968249,43,844,1,0,0,0,1,0,0,0.002332,0.00000,2,1,"(40.8, 52.2]"
34565,Miss Jasmine Russell,7644.8,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.20412,0,0.0,0,19,0.0,0,8.554158,61,652,0,1,0,0,0,1,0,0.013120,0.30103,2,1,"(52.2, 63.6]"
13388,Mr. Eric Berry,4554.8,99.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.20412,0,0.0,0,20,0.0,1,5.756517,39,808,0,1,0,0,1,0,0,0.007124,0.00000,1,1,"(29.4, 40.8]"
5366,Mr. Boris Reid,6065.1,75.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.20412,0,0.0,0,19,0.0,1,4.363612,48,343,0,0,0,1,0,1,0,0.012722,0.00000,2,1,"(40.8, 52.2]"
31409,Mr. Christian Allan,3746.5,35.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.20412,0,0.0,0,20,0.0,1,7.756011,38,944,1,0,0,0,0,0,0,0.008216,0.00000,1,1,"(29.4, 40.8]"


In [188]:
# será esta a abordagem mais acertada, isto é, assumir que todas as compras deste cliente foram "OfferPurchases"?

df.loc[(df['NumOfferPurchases'] > df[['NumAppPurchases','NumTakeAwayPurchases','NumStorePurchases']].sum(axis=1)),'NumOfferPurchases'] = df['NumAppPurchases'] + df['NumTakeAwayPurchases'] + df['NumStorePurchases']

In [189]:
df[(df['NumOfferPurchases'] > df[['NumAppPurchases','NumTakeAwayPurchases','NumStorePurchases']].sum(axis=1))] # confirmação do ajuste

,Name,Income,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Gender,MntTotal,Age,daysAsCardClient,Graduation,HighSchool,Master,PhD,Married,Single,Widow,Mnt_pday_card,Response_Campaigns,Total_Kids,has_Kids,age_bins
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## Data Review

Ver a dataframe no seu estado final
Drop: Id, name, birthyear, date_adherence, total_kids, mntTotal

In [190]:
df.head()

,Name,Income,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Gender,MntTotal,Age,daysAsCardClient,Graduation,HighSchool,Master,PhD,Married,Single,Widow,Mnt_pday_card,Response_Campaigns,Total_Kids,has_Kids,age_bins
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14594,Miss Leah Lewis,113947.0,43.0,3.666612,3.529045,3.755951,3.350442,2.750508,2.190332,0.477121,10,0.903090,7,4,0.0,0,19.242890,42,822,1,0,0,0,0,1,0,0.023410,0.30103,0,0,"(40.8, 52.2]"
19070,Miss Grace Black,114297.0,74.0,3.850095,3.565966,3.882012,3.265054,3.086716,1.672098,0.301030,5,1.000000,12,2,0.0,0,19.321940,49,731,1,0,0,0,1,0,0,0.026432,0.00000,0,0,"(40.8, 52.2]"
29888,Mr. Gavin Morrison,115476.0,85.0,3.616055,2.580925,3.689398,3.561221,2.580925,1.462398,0.301030,5,0.778151,7,1,0.0,1,17.490921,63,454,0,0,0,1,1,0,0,0.038526,0.00000,0,0,"(52.2, 63.6]"
12221,Mr. Ryan Glover,101172.0,25.0,3.555215,0.000000,3.248219,1.908485,2.257679,1.176091,0.778151,8,0.602060,8,5,0.0,1,12.145689,43,546,1,0,0,0,1,0,0,0.022245,0.00000,2,1,"(40.8, 52.2]"
21107,Mr. Paul Anderson,123128.0,64.0,3.731669,3.051924,3.977541,2.817565,2.750508,1.875061,0.000000,4,0.903090,8,5,0.0,1,18.204269,52,851,0,1,0,0,0,1,0,0.021392,0.60206,0,0,"(40.8, 52.2]"


In [191]:
df.describe()

,Income,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain,Gender,MntTotal,Age,daysAsCardClient,Graduation,HighSchool,Master,PhD,Married,Single,Widow,Mnt_pday_card,Response_Campaigns,Total_Kids,has_Kids
count,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000
mean,76141.811069,49.202886,3.022627,2.030261,2.968492,2.029535,2.027266,1.357073,0.470617,6.095729,0.600230,5.865195,5.354445,0.003061,0.733004,13.452468,46.534282,632.594567,0.502324,0.095439,0.162987,0.213829,0.653690,0.220366,0.034282,0.023505,0.094455,0.932162,0.704678
std,30999.688896,28.856545,0.838659,1.075018,0.717287,1.082055,1.079144,0.549658,0.207293,2.682044,0.247088,3.253601,2.705270,0.030203,0.442422,4.196910,11.992327,202.920394,0.500031,0.293841,0.369380,0.410037,0.475828,0.414524,0.181967,0.010890,0.204300,0.752672,0.456221
min,2493.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.968249,18.000000,286.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002332,0.000000,0.000000,0.000000
25%,51266.500000,24.000000,2.416641,1.612784,2.382017,1.612784,1.612784,1.000000,0.301030,4.000000,0.301030,3.000000,3.000000,0.000000,0.000000,10.308203,38.000000,457.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016012,0.000000,0.000000,0.000000
50%,76546.500000,49.000000,3.272074,2.257679,3.039811,2.257679,2.257679,1.397940,0.477121,6.000000,0.602060,5.000000,6.000000,0.000000,1.000000,13.812267,46.000000,627.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.021120,0.000000,1.000000,1.000000
75%,101052.750000,74.000000,3.707655,2.839792,3.558678,2.845718,2.833783,1.770852,0.602060,8.000000,0.778151,8.000000,7.000000,0.000000,1.000000,17.253992,56.000000,808.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.028800,0.000000,1.000000,1.000000
max,153703.000000,99.000000,4.175541,3.599992,4.160198,3.599992,3.599992,2.397940,1.230449,13.000000,1.397940,13.000000,20.000000,0.301030,1.000000,20.776802,75.000000,985.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.066812,1.204120,4.000000,1.000000


In [192]:
df_train= df.copy()
df_train.drop(['Name'], axis = 1, inplace = True)

In [193]:
df_train.isna().sum()

Income                   0
Recency                  0
MntMeat&Fish             0
MntEntries               0
MntVegan&Vegetarian      0
MntDrinks                0
MntDesserts              0
MntAdditionalRequests    0
NumOfferPurchases        0
NumAppPurchases          0
NumTakeAwayPurchases     0
NumStorePurchases        0
NumAppVisitsMonth        0
Complain                 0
Gender                   0
MntTotal                 0
Age                      0
daysAsCardClient         0
Graduation               0
HighSchool               0
Master                   0
PhD                      0
Married                  0
Single                   0
Widow                    0
Mnt_pday_card            0
Response_Campaigns       0
Total_Kids               0
has_Kids                 0
age_bins                 0
dtype: int64

## Data scaling
min max: income, recency, mnt..., purchases ..., age, daysasClient, mnt per ...

In [194]:
scaler = MinMaxScaler()
df_train = pd.DataFrame(scaler.fit_transform(df_train))
#df_train.describe()

TypeError: float() argument must be a string or a number, not 'pandas._libs.interval.Interval'

## PCA


A implementação do pca acima pareceu me estranha. deixo aqui outra e quando reunirmos vemos

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
df_train2= df_train.copy()
pca.fit(df_train2)
var= pca.explained_variance_ratio_
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

plt.title("PCA Variance against num of Componmnets")
plt.ylabel("Variance %")
plt.xlabel("Number of componments")
l = plt.axhline(80, color="red")

plt.plot(var1)
plt.grid()

In [ ]:
pca = PCA(n_components=7)
pca_train=pca.fit_transform(df_train2)
pca_train

In [ ]:
df_train

7 variaveis explicam ~80% da variancia

## Loading Scores for each PC

##  Correlação entre PC's e as variáveis originais

## Model Train

In [ ]:
from sklearn.cluster import KMeans

kmeans= KMeans(n_clusters = 100, max_iter =10000, random_state= 1)
kmeans.fit(pca_train)
pca_train_label = kmeans.labels_

In [ ]:
kmeans2= KMeans(n_clusters = 100, max_iter = 10000, random_state = 1)
kmeans2.fit(df_train)
df_train_label = kmeans.labels_

In [ ]:
dfTF= pd.DataFrame()
dfTF['Compare'] = pca_train_label== df_train_label
#pd.DataFrame(dfTF.groupby('Compare'))
dfTF.groupby(['Compare']).size().reset_index(name='counts') #TODO perguntar sobre pca e nao pca dar modelos iguais


### 3.2.2. Misclassifications

### 3.2.4. Binning

### 3.2.5. Reclassification

### 3.2.6. Power Transform

## 3.3. Data Reduction

### 3.3.1. Multicollinearity - Check correlation

### 3.3.2. Unary Variables

### 3.3.3. Variables with a high percentage of missing values

## 3.2. Back to Data Transformation

### 3.2.7. Apply ordinal encoding and create Dummy variables

### 3.2.8. Scaling